In [18]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf

from tensorflow import keras
print(tf.__version__)


2.0.0


In [19]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

# print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)


(20640, 8)
(20640,)


In [20]:
from sklearn.model_selection import train_test_split

x_train_all,x_test,y_train_all,y_test=train_test_split(housing.data, housing.target,random_state=7)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_all,y_train_all,random_state=11)

In [21]:
from sklearn.preprocessing import StandardScaler
transfer = StandardScaler()
x_train_scaled = transfer.fit_transform(x_train)
x_test_scaled = transfer.transform(x_test)
x_valid_scaled = transfer.transform(x_valid)

In [22]:
output_dir = 'data/generate_csv'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
def save_to_csv(output_dir, data , name_prefix, header=None,n_parts = 10):
    path_format = os.path.join(output_dir, "{}_{:02d}.csv")
#     path_format = os.path.join(output_dir, "{}/{:02d}.csv")


    filenames = []
    for file_idx,row_indices in enumerate(
            np.array_split(np.arange(len(data)),n_parts)):
        
        part_csv = path_format.format(name_prefix,file_idx)
        filenames.append(part_csv)
        with open(part_csv,'wt',encoding='utf-8') as f:
            if header is not None:
                f.write(header+'\n')
            for row_index in row_indices:
                f.write(','.join(
                [repr(col) for col in data[row_index]]
                ))
                f.write('\n')
    return filenames
        
    
    pass
train_data = np.c_[x_train_scaled, y_train]
test_data = np.c_[x_test_scaled, y_test]
valid_data = np.c_[x_valid_scaled, y_valid]
header_cols = housing.feature_names+["midianHouseValue"]
header_str = ",".join(header_cols)

train_filenames = save_to_csv(output_dir,train_data,'train',header_str,n_parts=20)
valid_filenames = save_to_csv(output_dir,valid_data,'valid',header_str,n_parts=10)
test_filenames = save_to_csv(output_dir,test_data,'test',header_str,n_parts=10)






FileNotFoundError: [Errno 2] No such file or directory: 'data/generate_csv/train/00.csv'

In [ ]:
print(housing.feature_names)

In [ ]:
import pprint
pprint.pprint(train_filenames)
pprint.pprint(valid_filenames)
pprint.pprint(test_filenames)

In [ ]:
# filename -> dataset
# read file -> dataset -> datasets -> merge
filename_dataset = tf.data.Dataset.list_files(train_filenames)
for filename in filename_dataset:
    print(filename)

In [ ]:
n_raaders = 5
dataset = filename_dataset.interleave(
    lambda filename: tf.data.TextLineDataset(filename).skip(1),#跳一行，省略header
    cycle_length= n_raaders
)
for line in dataset.take(15):
    print(line.numpy)


In [ ]:
# rf.io.decode_csv(str, record_defaults)
sample_str = '1,2,3,4,5'
record_defaults = [tf.constant(0,dtype=tf.int32)] *5
parsed_fields = tf.io.decode_csv(sample_str, record_defaults)
print(parsed_fields)

In [ ]:
# rf.io.decode_csv(str, record_defaults)
sample_str = '1,2,3,4,5'
record_defaults = [tf.constant(0,dtype=tf.int32),
                           0,
                           np.nan,
                           'hello',
                           tf.constant([])
                  ]
                            
parsed_fields = tf.io.decode_csv(sample_str, record_defaults)
print(parsed_fields)

In [ ]:
try:
    parsed_field = tf.io.decode_csv(',,,', record_defaults)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

In [ ]:
try:
    parsed_field = tf.io.decode_csv('1,2,3,4,5,5,6', record_defaults)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

In [6]:
 def parse_csv_line(line, n_fields=9):
        defs = [tf.constant(np.nan)]*n_fields
        parsed_fields = tf.io.decode_csv(line,record_defaults=defs)
        x = tf.stack(parsed_fields[0:-1])
        y = tf.stack(parsed_fields[-1:])
        return x,y
    
    
parse_csv_line('-1.1199749330438333,-1.329843308393715,0.1419004518620726,0.4658136987980791,-0.10301777467500105,-0.10744184416176107,-0.7950524078397521,1.530471676340,0.6674569',
               n_fields = 9)



(<tf.Tensor: id=11, shape=(8,), dtype=float32, numpy=
 array([-1.119975  , -1.3298433 ,  0.14190045,  0.4658137 , -0.10301778,
        -0.10744184, -0.7950524 ,  1.5304717 ], dtype=float32)>,
 <tf.Tensor: id=12, shape=(1,), dtype=float32, numpy=array([0.6674569], dtype=float32)>)

In [8]:
#1. filename -> dataset
#2. read file -> dataset -> datasets -> merge
#3. parse csv
def csv_resder_dataset(filenames, n_readers=5,
                                  batch_size =32, n_parse_threads=5,
                                shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat()
    dataset = dataset.interleave(
            lambda filename: tf.data.TextLineDataset(filename).skip(1),
            cycle_length = n_readers
                )
    dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(parse_csv_line, 
                          num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset

train_set = csv_resder_dataset(train_filenames,
                               batch_size=3)
import pprint
for x_batch, y_batch in train_set.take(2):
    
    print('x:')
    pprint.pprint(x_batch)
    print('y')
    pprint.pprint(y_batch)
    

x:
<tf.Tensor: id=182, shape=(3, 8), dtype=float32, numpy=
array([[ 0.04326301, -1.0895426 , -0.38878718, -0.10789865, -0.68186635,
        -0.0723871 , -0.8883662 ,  0.8213992 ],
       [-0.82195884,  1.8741661 ,  0.1821235 , -0.03170019, -0.6011179 ,
        -0.14337493,  1.0852206 , -0.8613995 ],
       [-0.66722274, -0.04823952,  0.34529406,  0.53826684,  1.8521839 ,
        -0.06112538, -0.8417093 ,  1.5204847 ]], dtype=float32)>
y
<tf.Tensor: id=183, shape=(3, 1), dtype=float32, numpy=
array([[1.426],
       [1.054],
       [1.59 ]], dtype=float32)>
x:
<tf.Tensor: id=184, shape=(3, 8), dtype=float32, numpy=
array([[ 0.63636464, -1.0895426 ,  0.09260903, -0.20538124,  1.2025671 ,
        -0.03630123, -0.6784102 ,  0.18223535],
       [ 2.5150437 ,  1.0731637 ,  0.5574401 , -0.17273512, -0.6129126 ,
        -0.01909157, -0.5710993 , -0.02749031],
       [-0.7543417 , -0.92934215, -0.92127204,  0.12428068, -0.598396  ,
        -0.18494336, -0.81838083,  0.85136   ]], dtype=float32)>

In [10]:
batch_size = 32
train_set = csv_resder_dataset(train_filenames,
                                            batch_size = batch_size)
valit_set = csv_resder_dataset(valid_filenames,
                                            batch_size = batch_size)
test_set = csv_resder_dataset(test_filenames,
                                            batch_size = batch_size)

In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(30,activation='relu',input_shape=[8]),
    keras.layers.Dense(1),
                 ])
model.compile(loss='mean_squared_error',
             optimizer = 'nadam',# SGD -> SGDM -> NAG ->AdaGrad -> AdaDelta -> Adam -> Nadam
             )
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 30)                270       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [14]:
callbacks = [keras.callbacks.EarlyStopping(patience=5,min_delta=1e-2)]
history = model.fit(train_set,epochs=5,
                    steps_per_epoch = 11160//batch_size,
                    validation_steps = 3870//batch_size,
                  validation_data=valit_set, 
                  callbacks= callbacks )

Train for 348 steps, validate for 120 steps
Epoch 1/5
348/348 [==============================] - 6s 17ms/step - loss: 1.3302 - val_loss: 0.6988
Epoch 2/5
348/348 [==============================] - 3s 9ms/step - loss: 0.5684 - val_loss: 0.5574
Epoch 3/5
348/348 [==============================] - 3s 9ms/step - loss: 0.4767 - val_loss: 0.4787
Epoch 4/5
348/348 [==============================] - 3s 9ms/step - loss: 0.4267 - val_loss: 0.4409
Epoch 5/5
348/348 [==============================] - 3s 9ms/step - loss: 0.4172 - val_loss: 0.4201


In [16]:
model.evaluate(test_set, steps = 5160//batch_size)

161/161 [==============================] - 1s 6ms/step - loss: 0.4255


0.42547776536171483